In [2]:
import numpy as np
import yaml
import json
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import gridspec
from caesar.logger.logger import setup_logger
from pathlib import Path
from typing import List, Dict, Any
from scipy.stats import gaussian_kde
from typing import List, Tuple
import numpy.typing as npt
from matplotlib import rcParams

config = {
    "font.family":'serif',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

In [4]:
file_path="./i4mcm/lmps-10000.dat"

In [5]:
with open(file_path,'r') as f:
    data_tmp=f.readlines()
data_tmp=[i.strip().split() for i in data_tmp]

In [6]:
data_df = pd.DataFrame(data_tmp, columns=[
                       'Step', 'Temp', 'PotEng', 'KinEng', 'TotEng', 'Press', 'Volume', 'Lx', 'Ly', 'Lz', 'Xy', 'Xz', 'Yz'])
data_df

,Step,Temp,PotEng,KinEng,TotEng,Press,Volume,Lx,Ly,Lz,Xy,Xz,Yz
0,0,0,-39740.211,0,-39740.211,48413.52,59812.539,27.611648,27.611648,78.452643,0,0,0
1,100,0.047703668,-39768.01,0.030824749,-39767.979,8498.2118,61065.279,27.803581,27.803573,78.993799,-2.5414708e-06,3.3851318e-05,-4.4634169e-05
2,200,0.00068460602,-39767.139,0.00044237288,-39767.138,9731.4157,61025.741,27.801817,27.801822,78.952633,-3.0899666e-05,-5.9205813e-05,3.1857219e-05
3,300,0.066686785,-39767.193,0.043091098,-39767.15,9913.7905,61015.127,27.793832,27.793835,78.984271,-7.424397e-05,0.00016429438,-8.7165357e-05
4,400,0.0066951783,-39767.36,0.0043262332,-39767.355,10037.881,61006.838,27.787576,27.78762,79.008987,4.7236294e-05,6.89387e-05,0.00019480073
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,399600,501.76002,-39450.76,324.223,-39126.537,9830.237,62455.154,28.058197,28.030057,79.411708,-0.009419566,-0.20826474,0.027746918
4016,399700,501.66685,-39457.978,324.16281,-39133.816,10336.703,62435.012,28.052455,28.062323,79.311054,0.029546208,0.24108182,-0.04376599
4017,399800,496.39891,-39457.233,320.75881,-39136.474,10921.284,62417.33,28.056011,28.057697,79.291612,-0.021875446,-0.22925763,-0.021080376
4018,399900,504.97655,-39454.764,326.30144,-39128.462,11284.199,62416.198,28.04952,28.023153,79.406283,0.012972721,0.16870907,0.013097184


In [7]:
temp_lattice=data_df[['Temp','Lx','Ly','Lz']].astype(float) # 提取温度和晶格常数
temp_lattice

,Temp,Lx,Ly,Lz
0,0.000000,27.611648,27.611648,78.452643
1,0.047704,27.803581,27.803573,78.993799
2,0.000685,27.801817,27.801822,78.952633
3,0.066687,27.793832,27.793835,78.984271
4,0.006695,27.787576,27.787620,79.008987
...,...,...,...,...
4015,501.760020,28.058197,28.030057,79.411708
4016,501.666850,28.052455,28.062323,79.311054
4017,496.398910,28.056011,28.057697,79.291612
4018,504.976550,28.049520,28.023153,79.406283


In [11]:
lattice_arr = []
l_x = []
l_y = []
l_z = []
for i in range(0, 300+1, 25):
    filtered_data = temp_lattice[(
        temp_lattice['Temp'] > i-5) & (temp_lattice['Temp'] < i+5)]  # 过滤温度
    lattice_avg = list(filtered_data[['Lx', 'Ly', 'Lz']].mean())  # 计算平均晶格常数
    l_x.append(lattice_avg[0])
    l_y.append(lattice_avg[1])
    l_z.append(lattice_avg[2])
lattice_avg = [l_x, l_y, [i/3 for i in l_z]]
lattice_avg

[[27.786520904761908,
  27.861017352272732,
  27.87630846938776,
  27.88419541919192,
  27.894210954545454,
  27.898643346733667,
  27.909667547738692,
  27.92008211167512,
  27.928589885416667,
  27.9410012460733,
  27.95126603296703,
  27.96335504571428,
  27.9721234244186],
 [27.786461476190475,
  27.86161134659091,
  27.876732362244894,
  27.883969777777782,
  27.892396,
  27.897123557788944,
  27.908890502512566,
  27.91819987817259,
  27.93038375520833,
  27.940507335078532,
  27.951956016483514,
  27.96308644,
  27.973024470930234],
 [26.325940158730152,
  26.2524633030303,
  26.24774630102041,
  26.264663156565657,
  26.27886341582492,
  26.301700777219427,
  26.314730475711894,
  26.327416223350255,
  26.338432116319442,
  26.347848403141366,
  26.356844446886445,
  26.364858533333333,
  26.376001176356592]]

In [36]:
filtered_data=temp_lattice[(temp_lattice['Temp']>0-5) & (temp_lattice['Temp']<0+5)] # 过滤温度异常值
lattice_avg=filtered_data[['Lx','Ly','Lz']].mean() # 计算平均晶格常数
list(lattice_avg)

[39.36737010000001, 39.36899176666667, 39.36744913333334]